# lammps_utility

This notebook demonstrates an example use case of lammps_utility

## dump_reader

First, `dump_reader` will be used to perform some elementary analysis on `example.dump`.

In [1]:
# Silence warnings to clean up notebook
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import Snapshot and Snapshots. No other entry points are needed for dump_reader
from lammps_utility.dump_reader import Snapshot, Snapshots
import numpy as np

In [3]:
snapshots = Snapshots.from_dump("example.dump")
print(snapshots)

Number of snapshots: 33
Timesteps: 0, 10000, 20000, ...,             310000, 320000
Custom data: disl_pos, disl_trajec


As we can see, custom data has been loaded from this dump file. We can access it on the `Snapshots` level with:

In [4]:
snapshots.custom["disl_pos"]

array([ -19.58377566,  -24.51401285,  -32.22488853,  -24.47504063,
        -20.95273182,  -23.43173126,  -40.88327053,  -31.91847058,
         -8.56145702,   28.56028607,   69.73070641,  117.04954071,
        -86.71281253,  -33.48045535,   25.07689473,   85.82095852,
       -108.01774047,  -64.34162962,  -14.66720144,   42.42353674,
         93.20935453, -110.39093178,  -64.52802911,  -18.52616722,
         27.4353135 ,   80.70320775,  125.47278381,  -78.23965326,
        -23.67241083,   33.57989977,   78.09682207, -125.47033538,
        -76.65729894])

Or on the `Snapshot` level with:

In [5]:
snapshots[0].custom["disl_pos"]

-19.5837756593944

For now, let's delete the custom data and re-calculate it:

In [6]:
del snapshots.custom["disl_pos"]
del snapshots.custom["disl_trajec"]
snapshots.custom

{}

There is no need here, but note the slicing capabilities of `Snapshots` objects:

In [7]:
print(snapshots.new[::5])

Number of snapshots: 7
Timesteps: 0, 50000, 100000, ...,             250000, 300000



Here is an example of converting a `Snapshot` object to an atomman `System` object to perform atomic manipulation, then converting back to a `Snapshot`

In [8]:
def trim_snapshot(snapshot, thickness = 20):
    system = snapshot.to_atomman()
    
    slab_center_y = system.box.ly/4
    
    is_in_slab = np.abs(system.atoms.pos[:, 1] - slab_center_y) <= (thickness/2)
    
    trimmed_system = system.atoms_ix[is_in_slab]
    
    return Snapshot.from_atomman(trimmed_system, timestep = snapshot.timestep)

Let's use `Ovito` to see the effect. This dump file is actually already trimmed to reduce the input file's size.

In [ ]:
trim = trim_snapshot(snapshots[0])   

trim.render()

Let's apply the modification to all `Snapshot` objects

In [9]:
trimmed_snapshots = Snapshots.empty()

for snapshot in snapshots:
    trimmed_snapshots += trim_snapshot(snapshot)

Let's perform some analysis to determine the dislocation position in each snapshot. First, let's initialize a new custom data entry:

In [10]:
trimmed_snapshots.custom["disl_pos"] = np.NaN
trimmed_snapshots.custom

{'disl_pos': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan])}

Now, lets loop over each snapshot and perform atomic analysis

In [11]:
for snapshot in trimmed_snapshots:
    system = snapshot.to_atomman()
    
    disl_atom_index = system.atoms.c_centro_atom.argmax()
    
    disl_position = system.atoms.pos[disl_atom_index, 0]
    
    snapshot.custom["disl_pos"] = disl_position
    
trimmed_snapshots.custom["disl_pos"]

array([ -19.58377566,  -24.51401285,  -32.22488853,  -24.47504063,
        -20.95273182,  -23.43173126,  -40.88327053,  -31.91847058,
         -8.56145702,   28.56028607,   69.73070641,  117.04954071,
        -86.71281253,  -33.48045535,   25.07689473,   85.82095852,
       -108.01774047,  -64.34162962,  -14.66720144,   42.42353674,
         93.20935453, -110.39093178,  -64.52802911,  -18.52616722,
         27.4353135 ,   80.70320775,  125.47278381,  -78.23965326,
        -23.67241083,   33.57989977,   78.09682207, -125.47033538,
        -76.65729894])

`unwrap` accounts for periodic boundaries so that the dislocation position increases in magnitude

In [12]:
def unwrap(positions, length):
    delta = np.diff(positions)
    delta_wrap = delta - np.sign(delta)*length
    displacements = np.where(abs(delta) < abs(delta_wrap), delta, delta_wrap)
    
    new_positions = positions.copy()
    
    for i, displacement in enumerate(displacements):
        new_positions[i + 1] = new_positions[i] + displacement
    
    return new_positions

Make another custom data entry and store

In [13]:
lx = trimmed_snapshots[0].box.lx

trimmed_snapshots.custom["disl_trajec"] = unwrap(trimmed_snapshots.custom["disl_pos"], lx)
trimmed_snapshots.custom["disl_trajec"]

array([ -19.58377566,  -24.51401285,  -32.22488853,  -24.47504063,
        -20.95273182,  -23.43173126,  -40.88327053,  -31.91847058,
         -8.56145702,   28.56028607,   69.73070641,  117.04954071,
        171.37677052,  224.6091277 ,  283.16647778,  343.91054157,
        408.16142563,  451.83753648,  501.51196466,  558.60270284,
        609.38852063,  663.87781737,  709.74072004,  755.74258193,
        801.70406266,  854.9719569 ,  899.74153296,  954.11867894,
       1008.68592137, 1065.93823197, 1110.45515427, 1164.97757987,
       1213.79061631])

Write new dumps to file. The custom data will be written aswell and reloaded into `custom` next time we load it in.

In [14]:
trimmed_snapshots.write_dump("analyzed.dump", allow_overwrite = True)

Let's use the plotting functions in `thermo_reader` for an interative plot.

In [15]:
from lammps_utility.thermo_reader import plot_data

plot_data(x = trimmed_snapshots.timesteps, y = trimmed_snapshots.custom["disl_trajec"], x_label = "Time (ps)", y_label = "Dislocation Trajectory (Å)")

# Plot opens in new window

## thermo_reader

The main function of `thermo_reader` is to parse YAML thermodynamic tables from LAMMPS log files. Let's do an example:

In [16]:
from lammps_utility.thermo_reader import parse_log_file

dataframes = parse_log_file("example.log")
dataframes[1]

,Step,Time,Temp,Press,PotEng,KinEng,Volume,Lx,Ly,Lz,Xy,Xz,Yz,Pxx,Pyy,Pzz,Pxy,Pxz,Pyz
0,0,0.00,0,-0.019003,-811399.68553,0,4.010520e+06,257.93127,904.909944,17.182695,0,0,0,-0.019003,-0.019003,-0.019003,4.981978e-11,-7.327799e-12,-1.483132e-12
1,50,0.05,0,-0.019003,-811399.68553,0,4.010520e+06,257.93127,904.909944,17.182695,0,0,0,-0.019003,-0.019003,-0.019003,-3.463058e-13,-1.564612e-12,5.221236e-12
2,100,0.10,0,-0.019003,-811399.68553,0,4.010520e+06,257.93127,904.909944,17.182695,0,0,0,-0.019003,-0.019003,-0.019003,1.708820e-12,1.418578e-12,-2.209892e-11
3,150,0.15,0,-0.019003,-811399.68553,0,4.010520e+06,257.93127,904.909944,17.182695,0,0,0,-0.019003,-0.019003,-0.019003,1.302905e-12,1.874407e-12,-4.745965e-12
4,200,0.20,0,-0.019003,-811399.68553,0,4.010520e+06,257.93127,904.909944,17.182695,0,0,0,-0.019003,-0.019003,-0.019003,1.640695e-12,-2.781325e-12,1.235831e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,4800,4.80,0,-0.019003,-811399.68553,0,4.010520e+06,257.93127,904.909944,17.182695,0,0,0,-0.019003,-0.019003,-0.019003,5.336516e-13,-3.134953e-12,2.245939e-12
97,4850,4.85,0,-0.019003,-811399.68553,0,4.010520e+06,257.93127,904.909944,17.182695,0,0,0,-0.019003,-0.019003,-0.019003,5.279744e-13,-5.471794e-12,-2.867103e-13
98,4900,4.90,0,-0.019003,-811399.68553,0,4.010520e+06,257.93127,904.909944,17.182695,0,0,0,-0.019003,-0.019003,-0.019003,1.396577e-12,-1.233232e-12,-3.941433e-13
99,4950,4.95,0,-0.019003,-811399.68553,0,4.010520e+06,257.93127,904.909944,17.182695,0,0,0,-0.019003,-0.019003,-0.019003,7.948002e-13,-1.159767e-12,2.946985e-12


Dataframes is a one-indexed dictionary (for user-ease in counting) containing `Pandas` DataFrames of the thermodynamic tables corresponding to each run. Also, information about the units, timestep, and run style is recorded if it is available:

In [17]:
dataframes[1].attrs

{'type': 'minimization', 'unit_style': 'metal'}

In [18]:
dataframes[9].attrs

{'timestep': 0.001, 'type': 'dynamics', 'unit_style': 'metal'}

Also, if `time` was not outputted but `timestep` and `step` were, `time` will automatically be calculated and added to the DataFrame

Also, `thermo_reader` offers an interactive plotter which auto-fills units (if they are available):

In [19]:
from lammps_utility.thermo_reader import plot_log_data

plot_log_data(dataframes, 9, "Temp")
# Plot opens in new window

## data_gui

Data gui launches an interactive GUI allowing global dump or thermodynamic properties to be plotted. You may test it on the example files given.

In [ ]:
from lammps_utility.data_gui import launch
launch()